#### SVI+Mortality data

In [2]:
import pandas as pd
import geopandas as gpd

In [4]:
mortality = pd.read_excel('/home/h6x/git_projects/ornl-svi-data-processing/raw_data/HepVu_County_Opioid_Indicators_05DEC22.xlsx')
svi = gpd.read_file('/home/h6x/git_projects/ornl-svi-data-processing/raw_data/svi/2018/SVI2018_US_tract.gdb')

In [5]:
def preprocess_overdose_data(overdose_df):
    """Preprocess overdose data."""
    overdose_df['GEO ID'] = overdose_df['GEO ID'].astype(str)
    overdose_df['GEO ID'] = overdose_df['GEO ID'].apply(lambda x: x.zfill(5))
    return overdose_df

In [7]:
def preprocess_svi_data(us_svi, raw_variables):
    """Preprocess SVI data by removing invalid values and normalizing."""
    for variable in raw_variables:
        us_svi = us_svi[us_svi[variable] != -999.00]

    # for var in raw_variables:
    #     max_val = us_svi[var].max()
    #     min_val = us_svi[var].min()
    #     us_svi[var] = (us_svi[var] - min_val) / (max_val - min_val)
    
    return us_svi

In [8]:
raw_variables = [
        'EP_POV', 'EP_UNEMP', 'EP_PCI', 'EP_NOHSDP', 'EP_UNINSUR', 'EP_AGE65',
        'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 'EP_LIMENG', 'EP_MINRTY', 'EP_MUNIT',
        'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ'
    ]

In [9]:
mortality = preprocess_overdose_data(mortality)
svi = preprocess_svi_data(svi, raw_variables)

In [ ]:
# filter out the columns we need
mortality = mortality[['GEO ID', 'Narcotic Overdose Mortality Rate 2018']]

In [11]:
svi.head(2)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Length,Shape_Area,geometry
609,01,ALABAMA,AL,01001,Autauga,01001020100,"Census Tract 201, Autauga County, Alabama",3.790677,1923,253,...,0,0,178,79,9.3,4.1,1018,0.150082,0.000948,"MULTIPOLYGON (((-86.50916 32.47344, -86.50620 ..."
610,01,ALABAMA,AL,01001,Autauga,01001020300,"Census Tract 203, Autauga County, Alabama",2.065365,3476,433,...,0,1,119,81,3.4,2.2,1836,0.100175,0.000516,"MULTIPOLYGON (((-86.47087 32.47573, -86.46964 ..."


In [14]:
mortality.head(2)

,GEO ID,Narcotic Overdose Mortality Rate 2018
0,01001,10.9
1,01003,14.9


In [15]:
df = pd.merge(svi, mortality,left_on='STCNTY', right_on='GEO ID', how='left')

In [17]:
df.shape

(72173, 129)

In [16]:
df.head(3)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Length,Shape_Area,geometry,GEO ID,Narcotic Overdose Mortality Rate 2018
0,01,ALABAMA,AL,01001,Autauga,01001020100,"Census Tract 201, Autauga County, Alabama",3.790677,1923,253,...,178,79,9.3,4.1,1018,0.150082,0.000948,"MULTIPOLYGON (((-86.50916 32.47344, -86.50620 ...",01001,10.9
1,01,ALABAMA,AL,01001,Autauga,01001020300,"Census Tract 203, Autauga County, Alabama",2.065365,3476,433,...,119,81,3.4,2.2,1836,0.100175,0.000516,"MULTIPOLYGON (((-86.47087 32.47573, -86.46964 ...",01001,10.9
2,01,ALABAMA,AL,01001,Autauga,01001020400,"Census Tract 204, Autauga County, Alabama",2.464982,3831,337,...,108,100,2.8,2.6,1951,0.114106,0.000609,"MULTIPOLYGON (((-86.45394 32.49318, -86.44742 ...",01001,10.9


In [18]:
all_selected_columns = raw_variables + ['Narcotic Overdose Mortality Rate 2018', 'geometry','ST', 'STCNTY','ST_ABBR','FIPS']

In [19]:
# filter out the columns we need
df = df[all_selected_columns]

In [21]:
# rename col "Narcotic Overdose Mortality Rate 2018" to NOD
df = df.rename(columns={"Narcotic Overdose Mortality Rate 2018": "NOD"})

In [22]:
df.head(3)

,EP_POV,EP_UNEMP,EP_PCI,EP_NOHSDP,EP_UNINSUR,EP_AGE65,EP_AGE17,EP_DISABL,EP_SNGPNT,EP_LIMENG,...,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ,NOD,geometry,ST,STCNTY,ST_ABBR,FIPS
0,11.3,3.6,31580.0,9.4,9.3,12.7,21.2,17.6,8.6,0.8,...,17.3,0.4,0.8,0.0,10.9,"MULTIPOLYGON (((-86.50916 32.47344, -86.50620 ...",01,01001,AL,01001020100
1,15.0,3.3,22527.0,12.8,3.4,16.7,24.2,21.2,8.3,3.3,...,1.4,2.2,7.6,0.0,10.9,"MULTIPOLYGON (((-86.47087 32.47573, -86.46964 ...",01,01001,AL,01001020300
2,2.8,2.9,30527.0,6.0,2.8,23.5,20.5,19.6,6.3,1.4,...,0.0,0.8,1.3,0.0,10.9,"MULTIPOLYGON (((-86.45394 32.49318, -86.44742 ...",01,01001,AL,01001020400


In [23]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [24]:
output_path = '/home/h6x/git_projects/universal-experiment-lab/experiment_2_bym_modeling/bym_svi_opioid_data/svi_mortality_2018.shp'

In [25]:
gdf.to_file(output_path, driver='ESRI Shapefile')